In [ ]:
# Create a function to generate points at a specified interval along a line
def generate_points_along_line(line, interval):
    points = []
    length = line.length
    distance = 0
    while distance < length:
        point = line.interpolate(distance)
        points.append(point)
        distance += interval
    return gpd.GeoSeries(points)

def find_perpendicular_points(point, path_points, segment_length=10):
    # Find the index of the closest point on the path
    distances = np.sqrt(((path_points - point)**2).sum(axis=1))
    closest_index = np.argmin(distances)
    
    # Use the closest points to approximate the derivative
    if closest_index == 0:  # If the closest is the first point, take the next point to calculate the slope
        next_index = 1
    elif closest_index == len(path_points) - 1:  # If the closest is the last point, take the previous point
        next_index = closest_index - 1
    else:
        # Take the next or previous point which is closer to calculate the slope
        if distances[closest_index + 1] < distances[closest_index - 1]:
            next_index = closest_index + 1
        else:
            next_index = closest_index - 1
    
    # Calculate the slope of the path at the closest point
    dy = path_points[next_index, 1] - path_points[closest_index, 1]
    dx = path_points[next_index, 0] - path_points[closest_index, 0]
    
    if dx == 0:  # Avoid division by zero
        perp_slope = 0
    else:
        perp_slope = -dx / dy
    
    # Calculate the mid-point for the perpendicular line segment
    mid_x, mid_y = point
    
    # Calculate the dx and dy for the perpendicular segment
    if perp_slope == 0:  # Horizontal line
        dx_perp = 0
        dy_perp = segment_length / 2
    else:
        dx_perp = segment_length / (2 * np.sqrt(1 + perp_slope**2))
        dy_perp = perp_slope * dx_perp
    
    # Calculate the two ends of the perpendicular segment
    start_point = (mid_x - dx_perp, mid_y - dy_perp)
    end_point = (mid_x + dx_perp, mid_y + dy_perp)
    
    return start_point, end_point



# Download Tiles

In [ ]:
### WORKS WITH NO BOARDER!
import os
import glob
import subprocess

# # Specify the directory where your TIFF files are located
# tif_dir = r"C:\Users\magst\Desktop\LIDARPODS\OUTPUTLIDAR\FIRE\AUGUST"
# output_tif_hag = r"C:\Users\magst\Desktop\LIDARPODS\OUTPUTLIDAR\MERGED\AUGUST_hag.tif"
# output_tif_dtm = r"C:\Users\magst\Desktop\LIDARPODS\OUTPUTLIDAR\MERGED\AUGUST_DTM.tif"

# tif_dir = r"C:\Users\magst\Desktop\LIDARPODS\OUTPUTLIDAR\FIRE\BEAR"
# output_tif_hag = r"C:\Users\magst\Desktop\LIDARPODS\OUTPUTLIDAR\MERGED\BEAR_hag.tif"
# output_tif_dtm = r"C:\Users\magst\Desktop\LIDARPODS\OUTPUTLIDAR\MERGED\BEAR_DTM.tif"

# tif_dir = r"C:\Users\magst\Desktop\LIDARPODS\OUTPUTLIDAR\FIRE\BUSH"
# output_tif_hag = r"C:\Users\magst\Desktop\LIDARPODS\OUTPUTLIDAR\MERGED\BUSH_hag.tif"
# output_tif_dtm = r"C:\Users\magst\Desktop\LIDARPODS\OUTPUTLIDAR\MERGED\BUSH_DTM.tif"

# tif_dir = r"C:\Users\magst\Desktop\LIDARPODS\OUTPUTLIDAR\FIRE\CALFCANYON"
# output_tif_hag = r"C:\Users\magst\Desktop\LIDARPODS\OUTPUTLIDAR\MERGED\CALFCANYON_hag.tif"
# output_tif_dtm = r"C:\Users\magst\Desktop\LIDARPODS\OUTPUTLIDAR\MERGED\CALFCANYON_DTM.tif"

# tif_dir = r"C:\Users\magst\Desktop\LIDARPODS\OUTPUTLIDAR\FIRE\CAMERONPEAK"
# output_tif_hag = r"C:\Users\magst\Desktop\LIDARPODS\OUTPUTLIDAR\MERGED\CAMERONPEAK_hag.tif"
# output_tif_dtm = r"C:\Users\magst\Desktop\LIDARPODS\OUTPUTLIDAR\MERGED\CAMERONPEAK_DTM.tif"

# tif_dir = r"C:\Users\magst\Desktop\LIDARPODS\OUTPUTLIDAR\FIRE\DIXIE"
# output_tif_hag = r"C:\Users\magst\Desktop\LIDARPODS\OUTPUTLIDAR\MERGED\DIXIE_hag.tif"
# output_tif_dtm = r"C:\Users\magst\Desktop\LIDARPODS\OUTPUTLIDAR\MERGED\DIXIE_DTM.tif"

# tif_dir = r"C:\Users\magst\Desktop\LIDARPODS\OUTPUTLIDAR\FIRE\RAFAEL"
# output_tif_hag = r"C:\Users\magst\Desktop\LIDARPODS\OUTPUTLIDAR\MERGED\RAFAEL_hag.tif"
# output_tif_dtm = r"C:\Users\magst\Desktop\LIDARPODS\OUTPUTLIDAR\MERGED\RAFAEL_DTM.tif"

# tif_dir = r"C:\Users\magst\Desktop\LIDARPODS\OUTPUTLIDAR\FIRE\SALT1"
# output_tif_hag = r"C:\Users\magst\Desktop\LIDARPODS\OUTPUTLIDAR\MERGED\SALT1_hag.tif"
# output_tif_dtm = r"C:\Users\magst\Desktop\LIDARPODS\OUTPUTLIDAR\MERGED\SALT1_DTM.tif"

# tif_dir = r"C:\Users\magst\Desktop\LIDARPODS\OUTPUTLIDAR\FIRE\SUGAR"
# output_tif_hag = r"C:\Users\magst\Desktop\LIDARPODS\OUTPUTLIDAR\MERGED\SUGAR_hag.tif"
# output_tif_dtm = r"C:\Users\magst\Desktop\LIDARPODS\OUTPUTLIDAR\MERGED\SUGAR_DTM.tif"

# tif_dir = r"C:\Users\magst\Desktop\LIDARPODS\OUTPUTLIDAR\FIRE\TELEGRAPH"
# output_tif_hag = r"C:\Users\magst\Desktop\LIDARPODS\OUTPUTLIDAR\MERGED\TELEGRAPH_hag.tif"
# output_tif_dtm = r"C:\Users\magst\Desktop\LIDARPODS\OUTPUTLIDAR\MERGED\TELEGRAPH_DTM.tif"

hag_tifs = glob.glob(os.path.join(tif_dir, "*3dep-lidar-hag*.tiff"))

# Prepare the gdal_merge command for HAG
merge_command_hag = [
    "python",
    "C:\\Users\\magst\\anaconda3\\envs\\LIDAR\\Scripts\\gdal_merge.py",
    "-o", output_tif_hag,
    "-n", "-9999",
    "-a_nodata","-9999",
    
] + hag_tifs

# Run the gdal_merge command for HAG and capture the output
process_hag = subprocess.run(merge_command_hag, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)

# Check if the command for HAG was successful
if process_hag.returncode != 0:
    # An error occurred, print the error
    print("Error occurred while merging TIFF files HAG:")
    print(process_hag.stderr)
else:
    print("TIFF files merged successfully for HAG.")

# Find all TIFF files that contain 'hag' in the filename
dtm_tifs = glob.glob(os.path.join(tif_dir, "*3dep-lidar-dtm*.tiff"))

# Prepare the gdal_merge command for DTM
merge_command_dtm = [
    "python",
    "C:\\Users\\magst\\anaconda3\\envs\\LIDAR\\Scripts\\gdal_merge.py",
    "-o", output_tif_dtm,
    "-n", "-9999",  
    "-a_nodata","-9999",

] + dtm_tifs

# Run the gdal_merge command for DTM and capture the output
process_dtm = subprocess.run(merge_command_dtm, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)

# Check if the command for DTM was successful
if process_dtm.returncode != 0:
    # An error occurred, print the error
    print("Error occurred while merging TIFF files DTM:")
    print(process_dtm.stderr)
else:
    print("TIFF files merged successfully for DTM.")


# Points and parallel lines

In [5]:
# Import necessary libraries working for small section
import geopandas as gpd
from shapely.geometry import Polygon
import utm
import pandas as pd
import torch
from torch.utils.data import Dataset
from matplotlib.patches import Patch
import numpy as np
import torch
import os
import laspy
import gc
from torch.utils.data import DataLoader, Dataset
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
from scipy.spatial import distance
from shapely.geometry import LineString

# Define a function to generate points along a line
def generate_points_along_line(line, interval_distance):
    total_length = line.length
    num_points = int(total_length // interval_distance) + 1
    points = []
    for i in range(num_points):
        point = line.interpolate(i * interval_distance)
        points.append(point)
    return points

# Read the test POD boundary.shp
#POD_shapefile_path = "C:\\Users\\magst\\Desktop\\WeatherKit\\TestPODbndry_BUCK_2.shp"
POD_shapefile_path = r"A:\LIDARPODS\InputData\123456_PODs_new_dis_wgs.shp"

# Load the shapefile for the fire perimeter
#shapefile_path_FirePerimeter = "C:\\Users\\magst\\Desktop\\WeatherKit\\NIFC_2020_WFIGS_AllPerims_20231010_CAMPEAK.shp"
shapefile_path_FirePerimeter = r"A:\LIDARPODS\InputData\IndividualSHPFirePerimeter\CalfCanyon_2022_perimeter.shp"

# Specify the output shapefile path
output_shapefile_path_PTS = "A:\\LIDARPODS\\scrap\\output_point_CAMPEAK.shp"

# Export the GeoDataFrame to a shapefile
output_shp_path_PERPLINE = "A:\\LIDARPODS\\scrap\\perpendicular_lines_CAMPEAK.shp"

# Define the distance interval between points (100 meters in this case)
interval_distance = 30  # in meters

# Buffer distances (m)
buff = 550


perp_length=500


# Specify the threshold distance for fire perimeter label
threshold_distance = 550

################################################################################################
################################################################################################
################################################################################################
fire_perimeters = gpd.read_file(shapefile_path_FirePerimeter)

# Determine the UTM zone automatically
# Use the centroid of the first geometry to determine UTM zone
lon, lat = fire_perimeters.geometry.iloc[0].centroid.coords[0]
utm_zone = utm.from_latlon(lat, lon)[2]
utm_crs = f'EPSG:326{utm_zone}'
print(utm_crs)
# Reproject to UTM
fire_perimeters = fire_perimeters.to_crs(utm_crs)

# buffer dist
buffer_distance_outward = buff
buffer_distance_inward = -buff

# Create outward buffer
outward_buffer = fire_perimeters.geometry.buffer(buffer_distance_outward)

# Create inward buffer
inward_buffer = fire_perimeters.geometry.buffer(buffer_distance_inward)

buffer_distance_inward_breach = buffer_distance_inward - 300
inward_buffer_BREACH = fire_perimeters.geometry.buffer(buffer_distance_inward_breach)

# Combine buffers to create the final shape
combined_buffers = outward_buffer.difference(inward_buffer)

# Create a new GeoDataFrame
buffered_gdf = gpd.GeoDataFrame(geometry=combined_buffers, crs=fire_perimeters.crs)

################
################
################

print(buffered_gdf.crs)

test_pod = gpd.read_file(POD_shapefile_path) # From before

test_pod = test_pod.to_crs(buffered_gdf.crs)

# Buffer the extent of fire_perimeters by a small distance
buffered_fire_perimeters = fire_perimeters.buffer(buff)  # Adjust the buffer distance as needed

# Clip the test_pod shapefile to the buffered extent of fire_perimeters
test_pod = gpd.clip(test_pod, buffered_fire_perimeters)

# Ensure both GeoDataFrames are in the same CRS
test_pod = test_pod.to_crs(buffered_fire_perimeters.crs)

# Ensure both GeoDataFrames are in the same CRS
#test_pod = test_pod.to_crs(buffered_gdf.crs)

# Explode MultiPolygon geometries if they exist in test_pod
if any(test_pod.geometry.type == 'MultiPolygon'):
    test_pod = test_pod.explode(index_parts=False)

# # Then try clipping again
# held_pod = gpd.clip(buffered_gdf, test_pod)

# # Clip the buffered area with the second shapefile
# #held_pod = gpd.clip(buffered_gdf, test_pod)

# # Label the clipped area
# held_pod['label'] = 'held_POD'
# Use overlay to perform intersection, which is similar to clipping
held_pod = gpd.overlay(buffered_gdf, test_pod, how='intersection')

# Label the intersected area
held_pod['label'] = 'held_POD'


# Find the difference: the areas in buffered_gdf not covered by held_pod
#breached_pod = test_pod.geometry.difference(held_pod.unary_union)
#breached_pod_gdf = gpd.GeoDataFrame(geometry=breached_pod, crs=buffered_gdf.crs)
#breached_pod_gdf['label'] = 'breached_POD'

#breached_pod = gpd.clip(inward_buffer_BREACH, test_pod)
# Convert inward_buffer_BREACH to a GeoDataFrame if it's not already
if isinstance(inward_buffer_BREACH, gpd.GeoSeries):
    inward_buffer_BREACH_gdf = gpd.GeoDataFrame(geometry=inward_buffer_BREACH)
else:
    inward_buffer_BREACH_gdf = inward_buffer_BREACH

# Now use gpd.overlay with GeoDataFrames
breached_pod = gpd.overlay(inward_buffer_BREACH_gdf, test_pod, how='intersection')

#breached_pod = gpd.overlay(inward_buffer_BREACH, test_pod, how='intersection')


breached_pod_gdf = gpd.GeoDataFrame(geometry=breached_pod, crs=buffered_gdf.crs)
breached_pod_gdf['label'] = 'breached_POD'

# Generate points along held_pod geometry
held_pod_points = generate_points_along_line(held_pod.geometry.iloc[0], interval_distance)

# Generate points along breached_pod_gdf geometry
breached_pod_points = generate_points_along_line(breached_pod_gdf.geometry.iloc[0], interval_distance)

# Extract the boundaries of MultiPolygons in fire_perimeters
fire_perimeter_boundaries = fire_perimeters.boundary

# Generate points along the fire perimeter boundaries
fire_perimeter_points = []
for boundary in fire_perimeter_boundaries:
    points = generate_points_along_line(boundary, interval_distance)
    fire_perimeter_points.extend(points)

# Assuming you have 'held_pod_points', 'breached_pod_points', and 'fire_perimeter_points' GeoSeries objects
held_pod_df = gpd.GeoDataFrame(geometry=held_pod_points)
breached_pod_df = gpd.GeoDataFrame(geometry=breached_pod_points)
fire_perimeter_df = gpd.GeoDataFrame(geometry=fire_perimeter_points)

# Concatenate the three DataFrames
merged_df = pd.concat([held_pod_df, breached_pod_df, fire_perimeter_df])

# Create a new GeoSeries from the concatenated DataFrame
merged_geo_series = gpd.GeoSeries(merged_df['geometry'])
print(f"number of held, breached, and fire perimeter points: {len(merged_geo_series)}")

# Create a list of labels for 'held_POD', 'breached_POD', and 'fire_perimeter' GeoDataFrames
held_labels = [1] * len(held_pod_points)
breached_labels = [0] * len(breached_pod_points)
fire_labels = [2] * len(fire_perimeter_points)
labels = held_labels + breached_labels + fire_labels

# Data for the new label
data = {
    'labels': labels,
    'merged_geo_series_x': merged_geo_series.x,
    'merged_geo_series_y': merged_geo_series.y
}

df = pd.DataFrame(data)

df['labels'] = df['labels'].astype(np.int16)
df['merged_geo_series_x'] = df['merged_geo_series_x'].astype(np.float32)
df['merged_geo_series_y'] = df['merged_geo_series_y'].astype(np.float32)

minx, miny, maxx, maxy = test_pod.total_bounds
gdf_extent = [minx, maxx, miny, maxy]
print(gdf_extent)

# Plot the points and color them based on labels
plt.figure(figsize=(10, 8))

# Plot the fire perimeter polyline with a dashed line
fire_perimeters.plot(ax=plt.gca(), color='lightgray', linewidth=2, linestyle='--', label='Fire Perimeter', alpha=0.5)

# Plot the test_pod polyline
test_pod.plot(ax=plt.gca(), color='black', linewidth=2, label='Pod Boundary')

# Separate the points based on labels
held_points = df[df['labels'] == 1]
breached_points = df[df['labels'] == 0]
fire_points = df[df['labels'] == 2]

#########################################################
#########################################################
#########################################################
# Create a function to calculate the distance between two points
def calculate_distance(point1, point2):
    return distance.euclidean(point1, point2)

# Create NumPy arrays to store coordinates and labels
coordinates = df[['merged_geo_series_x', 'merged_geo_series_y']].values
labels = df['labels'].values

# Create an array to track labels to be removed
to_remove = np.zeros(len(coordinates), dtype=bool)

# Iterate through the DataFrame to find label 2 to be removed
for i in range(len(coordinates)):
    if labels[i] == 2:
        point1 = coordinates[i]
        close_to_label_0_1 = np.any(np.logical_and(labels != 2, np.linalg.norm(coordinates - point1, axis=1) <= threshold_distance))
        if close_to_label_0_1:
            to_remove[i] = True

# Remove the rows with labels 2 that are within 300 meters of a 0 or 1
filtered_coordinates = coordinates[~to_remove]
filtered_labels = labels[~to_remove]

# Create a new DataFrame
filtered_df = pd.DataFrame({
    'labels': filtered_labels,
    'merged_geo_series_x': filtered_coordinates[:, 0],
    'merged_geo_series_y': filtered_coordinates[:, 1]
})

# Convert the DataFrame 'df' to a GeoDataFrame
gdf = gpd.GeoDataFrame(filtered_df, geometry=gpd.points_from_xy(filtered_df['merged_geo_series_x'], filtered_df['merged_geo_series_y']), crs=utm_crs)

# Save the GeoDataFrame as a shapefile
gdf.to_file(output_shapefile_path_PTS)

#########################################################
#########################################################
#########################################################

plt.scatter(held_points['merged_geo_series_x'], held_points['merged_geo_series_y'], c='green', label='Held', marker='o')
plt.scatter(breached_points['merged_geo_series_x'], breached_points['merged_geo_series_y'], c='red', label='Breach', marker='x')
plt.scatter(filtered_df['merged_geo_series_x'][filtered_df['labels'] == 2], filtered_df['merged_geo_series_y'][filtered_df['labels'] == 2], c='blue', label='Fire Perimeter')

# Add labels and legend
plt.xlabel('Easting')
plt.ylabel('Northing')
plt.xlim(minx - 500, maxx + 500)
plt.ylim(miny - 500, maxy + 500)
plt.legend(loc='best')

#plt.savefig(r'C:\Users\magst\Desktop\LIDAR\FIGURES\overlayed.png', dpi=300, bbox_inches='tight')

plt.show()

print("done")

#
##
###
####
#####
######
#######

#converted_held_pod_points = np.array([(point.x, point.y) for point in held_pod_points])
#converted_breached_pod_points = np.array([(point.x, point.y) for point in breached_pod_points])
fireline_point = filtered_df[filtered_df['labels'] == 2]
converted_fireline_points = fireline_point[['merged_geo_series_x', 'merged_geo_series_y']].values

breach_point = filtered_df[filtered_df['labels'] == 0]
converted_breach_points = breach_point[['merged_geo_series_x', 'merged_geo_series_y']].values

held_point = filtered_df[filtered_df['labels'] == 1]
converted_held_points = held_point[['merged_geo_series_x', 'merged_geo_series_y']].values

# # Create an empty list to hold the LineString objects
# line_strings = []

test_pod_points = filtered_df[['merged_geo_series_x', 'merged_geo_series_y']].to_numpy()



# for point in np.vstack((converted_fireline_points, converted_breach_points, converted_held_points)):
#     start, end = find_perpendicular_points(point, test_pod_points, perp_length)
    
#     # Check if start and end points have valid coordinates (not NaN or infinite)
#     if np.all(np.isfinite(start)) and np.all(np.isfinite(end)):
#         line = LineString([start, end])
#         line_strings.append(line)

# # Create a GeoDataFrame with the line segments
# gdf = gpd.GeoDataFrame(geometry=line_strings)

# # Set a coordinate reference system (CRS) for the GeoDataFrame
# gdf.crs = utm_crs

# #If df['labels'] is the same length as gdf and corresponds to each line
# #Reset index to ensure proper alignment when adding labels
# filtered_df.reset_index(drop=True, inplace=True)
# gdf.reset_index(drop=True, inplace=True)

# #Add labels to your GeoDataFrame
# gdf['label'] = filtered_df['labels']


# gdf.to_file(output_shp_path_PERPLINE)
# print("Shapefile with labels saved.")


# Create an empty list to hold the LineString objects for fireline
fireline_line_strings = []

for point in converted_fireline_points:
    start, end = find_perpendicular_points(point, test_pod_points, perp_length)
    
    # Check if start and end points have valid coordinates (not NaN or infinite)
    if np.all(np.isfinite(start)) and np.all(np.isfinite(end)):
        line = LineString([start, end])
        fireline_line_strings.append(line)

# Create a GeoDataFrame for fireline
fireline_gdf = gpd.GeoDataFrame(geometry=fireline_line_strings)
fireline_gdf.crs = utm_crs
fireline_gdf['label'] = 2  # Assign label 2 for fireline

# Create an empty list to hold the LineString objects for breach
breach_line_strings = []

for point in converted_breach_points:
    start, end = find_perpendicular_points(point, test_pod_points, perp_length)
    
    # Check if start and end points have valid coordinates (not NaN or infinite)
    if np.all(np.isfinite(start)) and np.all(np.isfinite(end)):
        line = LineString([start, end])
        breach_line_strings.append(line)

# Create a GeoDataFrame for breach
breach_gdf = gpd.GeoDataFrame(geometry=breach_line_strings)
breach_gdf.crs = utm_crs
breach_gdf['label'] = 0  # Assign label 0 for breach

# Create an empty list to hold the LineString objects for held
held_line_strings = []

for point in converted_held_points:
    start, end = find_perpendicular_points(point, test_pod_points, perp_length)
    
    # Check if start and end points have valid coordinates (not NaN or infinite)
    if np.all(np.isfinite(start)) and np.all(np.isfinite(end)):
        line = LineString([start, end])
        held_line_strings.append(line)

# Create a GeoDataFrame for held
held_gdf = gpd.GeoDataFrame(geometry=held_line_strings)
held_gdf.crs = utm_crs
held_gdf['label'] = 1  # Assign label 1 for held

# Concatenate the GeoDataFrames for fireline, breach, and held
gdf = pd.concat([fireline_gdf, breach_gdf, held_gdf], ignore_index=True)

# Save the GeoDataFrame as a shapefile
gdf.to_file(output_shp_path_PERPLINE)
print("Shapefile with labels saved.")


#######
######
#####
####
###
##
#

EPSG:32613
EPSG:32613


C:\Users\admin-magstadt\AppData\Local\Temp\ipykernel_4724\3658775547.py:125: UserWarning: `keep_geom_type=True` in overlay resulted in 36 dropped geometries of different geometry types than df1 has. Set `keep_geom_type=False` to retain all geometries
  held_pod = gpd.overlay(buffered_gdf, test_pod, how='intersection')
C:\Users\admin-magstadt\AppData\Local\Temp\ipykernel_4724\3658775547.py:144: UserWarning: `keep_geom_type=True` in overlay resulted in 23 dropped geometries of different geometry types than df1 has. Set `keep_geom_type=False` to retain all geometries
  breached_pod = gpd.overlay(inward_buffer_BREACH_gdf, test_pod, how='intersection')


ValueError: Must pass array with one dimension only.

In [ ]:
pip install pygeos
